# LoopTree Tutorial

LoopTree is a model to evaluate the latency and energy of a fused-layer dataflow accelerator.

To model energy and latency, a workload, architecture, and mapping have to be specified. First, we discuss how these are specified. Then, we show how to run the LoopTree model.

## Specifying Architecture, Workload, and Mapping

For the LoopTree model to estimate energy and latency, the user must specify an architecture, workload, and mapping. Below, we discuss how to specify each of these inputs.

### Architecture
In LoopTree, the architecture of an accelerator is abstracted as buffers that use the Buffet semantics and computation units, following the [Timeloop v4 specification](https://timeloop.csail.mit.edu/v4).

An example architecture that we will use here is displayed below.

In [2]:
from util import *
from pprint import pp
show_config('flat/architecture.yaml')

variables:
 global_cycle_seconds: 1e-9 # 1 / 0.7 GHz = 1.43 ns
 technology: "45nm"

architecture:
 version: 0.4
 nodes:
 - !Component
   name: MainMemory
   class: DRAM            # off chip memory
   Attributes:		# do we have to set depth for off chip?
       width: 3200	# where do we find a value for this?
       word_bits: 16
       datawidth: 16
       shared_bandwidth: 200 # words per cycle @ 10^9 cycles/s = 400 GB/s bandwidth
   required_actions: ['read', 'write']
 - !Component
   name: GlobalBuffer
   class: SRAM           # on chip memory
   attributes:  # does the depth/width/blksz matter as long as the capacity is right?
       depth: 65536            # 4096 bits per row × 65536 rows = 32MB total
       width: 4096             # Number of bits per row (256 words × 16 bits)
       block_size: 256         # 256 words per row
       word_bits: 16           # Each word is 16 bits (2 bytes)
       datawidth: 16          
       n_rdwr_ports: 4         # Where do we find a value fo

### Workloads

DNN workloads in LoopTree are abstracted as a set of Einsums, each representing a layer.

Each Einsum is specified as a dictionary. Each Einsum reads/writes to a number of tensors (referred to as `data_spaces` in the specification), and each tensor is either an input to the operation or an output (indicated by a key "read_write" with value `True`). Moreover, a "projection" from the Einsum to each tensor, which describes the index of the element in the data that an operation in the Einsum accesses, must be specified. Finally, we specify must specify the *shape* of the Einsum, which is the bounds of its dimensions (also referred to as "ranks").

An example comprising two fully-connected layers are shown below.

In [3]:
show_config('flat/workload.yaml')

problem:
 - shape:
     name: QK
     dimensions: [ B1, H1, M1, P1, E1]
     data_spaces:
     - name: Q
       dimensions: [ Q_B, Q_H, Q_M, Q_E ]
       projection: '[ B1, H1, M1, E1 ]'
     - name: K
       dimensions: [ K_B, K_H, K_P, K_E ]
       projection: '[ B1, H1, P1, E1 ]'
     - name: L
       dimensions: [ L_B, L_H, L_M, L_P ]
       projection: '[ B1, H1, M1, P1 ]'
       read_write: True

   instance: >-
    0 <= B1 < 64 and
    0 <= H1 < 12 and
    0 <= M1 < 512 and
    0 <= P1 < 512 and
    0 <= E1 < 64


 - shape:
     name: SM
     dimensions: [ B2, H2, M2, P2 ]
     data_spaces:
     - name: L
       dimensions: [ L_B, L_H, L_M, L_P ]
       projection: '[ B2, H2, M2, P2 ]'
     - name: A
       dimensions: [ A_B, A_H, A_M, A_P ]
       projection: '[ B2, H2, M2, P2 ]'
       read_write: True

   instance: >-
    0 <= B2 < 64 and
    0 <= H2 < 12 and
    0 <= M2 < 512 and
    0 <= P2 < 512


 - shape:
     name: AV
     dimensions: [ B3, H3, M3, P3, F3 ]
     data_sp

In this workload specification, we specify two Einsums Fc1 and Fc2. Each Einsum has three dimensions. Fc1 has dimensions P1, M1, and C1; Fc2 has dimensions P2, M2, and C2. Finally, we specify the shape of the Einsums as bounds for each Einsum dimension (rank).

### Mapping

The LoopTree mapping is a tree-structure that contains nodes of the types described below.
- **Loops**: a loop node specifies a rank in the Einsum that is partitioned and the shape of the tiles that results from the partitioning. If the loop is a "temporal" loop, then the tiles are scheduled to be processed one at a time. If the loop is a "spatial" loop, then the tiles are scheduled to parallel hardware units.
- **Branching point**: nodes above a branching point (*i.e.*, ancestor nodes) describe inter-Einsum mapping, which is applied to all Einsums under that branching point. Nodes underneath the branching point (*i.e.*, within each branch) pertain only to the Einsum of that branch.
- **Storage**: a storage node specifies the tiles of tensors to retain and which hardware level retains the tiles.
- **Compute**: a compute node specifies the hardware level used to compute operations of an Einsum. This node has to be a leaf node, and it also denotes the Einsum that a particular branch pertains to.

First, we discuss a layer-by-layer mapping example.

In [4]:
show_config('flat/mapping.yaml')

type: fused
nodes:
-type: storage
target: 0 # MainMemory/DRAM
dspace: [Q, K, V, O] # Exclude L and A, to reduce memory traffic


-type: temporal # Tiling Q, K, V, L, A, O by size Bx, Hx, and Nx.
rank: B1
tile_shape: 256 # TODO - we need to define tile size
-type: temporal 
rank: H1
tile_shape: 256
-type: temporal 
rank: M1
tile_shape: 256 # - 256 # The goal of tiling

-type: storage # store tiles of all tensors in SRAM
target: 1 # GlobalBuffer/SRAM
dspace: [Q, K, V, O, L, A]

-type: sequential
branches:

# Now, branches for each layer (QK).
# The amount of data sent to the PEs must be <= meshX * meshY. 
- - type: temporal
    rank: B1
    tile_shape: 1 
- - type: temporal
    rank: H1
    tile_shape: 1 

# Bringing the weight-stationary matrix into the correct tile to be distributed through the PEs
- - type: temporal
    rank: E1
    tile_shape: 256
- - type: temporal
    rank: P1
    tile_shape: 256

# Qtile: Bx, Hx, Mx, Ex
# Ktile: Bx, Hx, Px, Ex
# Ltile: Bx, Hx, Mx, Px	

- type: spa

Explanation:
- (1.a) The storage node specifies that the tensors specified by the `dspace` key will be retained in target 0, which we will bind to MainMemory.
        Note that MainMemory retains Fmap2.
- (1.b) The sequential node specifies that the following branches (one for Fc1 and one for Fc2, as we will see shortly) are processed sequentially.
- (1.c) Filter1 is retained in GlobalBuffer
- (1.d) The rank P1 (which is a rank of the Fc1 Einsum) is partitioned into tiles with shape 1 and we will iterate over the tiles one at a time (temporal iteration).
- (1.e) Tiles of Fmap1 and Fmap2 are retained in GlobalBuffer. We retain tiles becuase the P1 rank has been partitioned.
- (1.f) Operations of Fc1 are processed in the MACC unit. Moreover, this node specifies that this branch is relevant to Fc2.
- (1.g) This and the following nodes specify how Fc2 is mapped.

In [5]:
show_config('fused.mapping.yaml')

mapping:
  type: fused
  nodes:
  - type: storage
    target: 0  # level 2 is bound to MainMemory
    dspace: [Filter1, Filter2, Fmap1, Fmap3] #-------- node 2.a
  - type: storage
    target: 1  # level 1 is bound to GlobalBuffer
    dspace: [Filter1, Filter2]
  - type: temporal #---------------------------------- node 2.b
    rank: P2
    tile_shape: 1
  - type: storage  #---------------------------------- node 2.c
    target: 1  # level 1 is bound to GlobalBuffer
    dspace: [Fmap1, Fmap2, Fmap3]
  - type: sequential  #------------------------------- node 2.d
    branches:
    - - type: temporal
        rank: C1
        tile_shape: 1
      - type: temporal
        rank: M1
        tile_shape: 1
      - type: compute
        einsum: Fc1
        target: 2  # level 0 is bound to MACC
    - - type: temporal
        rank: C2
        tile_shape: 1
      - type: temporal
        rank: M2
        tile_shape: 1
      - type: compute
        einsum: Fc2
        target: 2  # level 0 is bound to

Explanation:
- (2.a) Similar to (1.a), but note that Fmap2 is no longer retained in MainMemory.
- (2.b) In this mapping, the $P2$ rank is partitioned to create tiles that are rows of feature maps Fmap1, Fmap2, and Fmap3. The tile shape attribute of the loop node implies that the tiles of Fmap3 have shape 1 in the $P2$ rank. LoopTree infers the shape of the tiles of Fmap2 and Fmap1 based on what is required as inputs to compute the specified tile of Fmap3. In this case, tiles that each contains one row of Fmap2 is required to compute tiles of Fmap3, and tiles that each contains one row of Fmap1 is required to compute tiles of Fmap2 in turn.
- (2.c) The tiles of Fmap1, Fmap2, and Fmap3 are retained in GlobalBuffer.
- (2.d) Similar to before, Fc1 and Fc2 are processed sequentially. However, as we have put node (2.b) above this sequential node, *tiles* of Fc1 and Fc2 are processed sequentially in alternating fashion: a tile of Fc1 produces a tile of Fmap2, the tile of Fmap2 is consumed by a tile of Fc2, another tile of Fc1 is produced, and so on.

## Running the Model

Using the PyTimeloop library, running the model is as simple as follows.

In [2]:
from util import *
from pprint import pp

from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    # ['flat/architecture.yaml', 'flat/workload.yaml', 'flat/mapping.yaml'],
    ['architecture.yaml', 'flat/workload.yaml', 'fused.mapping.yaml'],
    # ['flat/architecture.yaml', 'two_fc.workload.yaml', 'fused.mapping.yaml'],
TMP_DIR,
    bindings,
    call_accelergy=True
)

IndexError: map::at

The result returned by the model contains several information, such as latency:

In [13]:
print('Latency:', stats.latency)

Latency: 1048576


and also energy, which is computed by calculating the number of actions to each hardware component and multiplying that with the energy per action estimated using the Accelergy tool:

In [13]:
print('Energy:')
pp(stats.energy)

Energy:
{('MainMemory', 'read'): 50331648.0,
 ('GlobalBuffer', 'read'): 380775448.57600003,
 ('GlobalBuffer', 'write'): 123521941.50400001,
 ('MainMemory', 'write'): 33554432.0,
 ('MACC', 'compute'): 886046.72}


Now, we compare these results with the fused mapping:

In [3]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

Latency: 1048576
Energy:
{('MainMemory', 'read'): 33554432.0,
 ('GlobalBuffer', 'read'): 380775448.57600003,
 ('GlobalBuffer', 'write'): 122579025.92,
 ('MainMemory', 'write'): 16777216.0,
 ('MACC', 'compute'): 886046.72}


As we can see, the energy consumption due to MainMemory reads and writes have decreased significantly from fusion.

Here, we have not modeled the impact of limited MainMemory bandwidth; thus, because the MACC unit utilization is the same (100%) in both cases, the latency is the same. However, if the layer-by-layer latency is limited by MainMemory bandwidth, fusion will also decrease latency.

In [15]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused-pipeline.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

Latency: 528384
Energy:
{('MainMemory', 'read'): 33554432.0,
 ('GlobalBuffer', 'read'): 380775448.57600003,
 ('GlobalBuffer', 'write'): 122579025.92,
 ('MainMemory', 'write'): 16777216.0,
 ('MACC', 'compute'): 886046.72}
